# 📊 FiscalBot Gemini – Análise Inteligente de CSV com IA

In [1]:
# ✅ Instalação das bibliotecas necessárias
!pip install -q pandas python-dotenv langchain langchain-google-genai google-generativeai rich

# 📦 Explicação das bibliotecas:
# - pandas: biblioteca para leitura e manipulação de dados estruturados (como arquivos CSV)
# - python-dotenv: permite carregar variáveis sensíveis como chaves de API de arquivos .env
# - langchain: framework para conectar modelos de linguagem (LLMs) a aplicações
# - langchain-google-genai: extensão do LangChain para uso com a API Gemini (Google)
# - google-generativeai: SDK oficial do Google para acesso direto ao modelo Gemini
# - rich: biblioteca para saída visual mais agradável e estilizada no terminal ou notebook


In [2]:
# ✅ Imports essenciais

# 🔒 Módulo padrão para interagir com variáveis de ambiente e sistema de arquivos
import os

# 📊 Biblioteca para leitura, manipulação e análise de dados tabulares (como CSVs)
import pandas as pd

# 🤖 SDK oficial da Google para acesso direto aos modelos da família Gemini
import google.generativeai as genai

# 🔐 Carregamento de variáveis do arquivo .env para dentro do ambiente do Python
from dotenv import load_dotenv

# 🔁 Integração do LangChain com a API do Gemini (GoogleGenerativeAI)
from langchain_google_genai import GoogleGenerativeAI

# 🧩 Componente de template do LangChain: estrutura o prompt com variáveis
from langchain.prompts import PromptTemplate

# 🔗 Cadeia LLM do LangChain: conecta o modelo LLM ao prompt de forma funcional
from langchain.chains import LLMChain

# 🎨 Biblioteca para exibir saídas estilizadas no terminal (texto colorido, painéis, etc.)
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

# 🖥️ Inicializa o console estilizado para exibição rica no terminal
console = Console()


c:\Users\Usuário\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ✅ Carregamento de variáveis de ambiente

# 🔄 Carrega as variáveis definidas no arquivo .env (como a chave da API Gemini) para o ambiente Python
load_dotenv()

# 🔐 Recupera a chave da API Gemini armazenada na variável de ambiente GEMINI_API_KEY
gemini_api_key = os.getenv("GEMINI_API_KEY")

# ⚠️ Verifica se a chave foi carregada corretamente
if not gemini_api_key:
    # ❌ Exibe uma mensagem de erro formatada caso a chave não seja encontrada
    console.print(Panel(
        "[red]❌ ERRO: Chave da API Gemini não encontrada no arquivo .env[/red]",
        title="FALHA"
    ))
else:
    # ✅ Exibe uma mensagem de sucesso com os primeiros e últimos 4 dígitos da chave, ocultando o restante
    console.print(Panel(
        f"[green]🔑 Chave carregada: {gemini_api_key[:4]}...{gemini_api_key[-4:]}[/green]",
        title="Chave OK"
    ))
# 🔐 Observação: Isso garante segurança (não exibe a chave completa) e usabilidade (informa visualmente o status do carregamento da chave).


╭─────────────────────────────────────────────────── Chave OK ────────────────────────────────────────────────────╮
│ 🔑 Chave carregada: AIza...4WgM                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
# ✅ Configuração do modelo Gemini via LangChain

# 🤖 Inicializa o modelo de linguagem (LLM) usando a API Gemini da Google, via LangChain
llm = GoogleGenerativeAI(
    model="models/gemini-1.5-flash",   # Modelo escolhido (leve, rápido e gratuito para contas padrão)
    google_api_key=gemini_api_key,     # Chave da API carregada anteriormente do arquivo .env
    temperature=0                      # Temperatura 0 = respostas mais determinísticas e precisas
)

# 🧠 Template de prompt: define como será estruturada a pergunta enviada ao modelo
template = """
Você é um analista fiscal inteligente.

Com base nos dados a seguir:

{contexto}

Responda à pergunta:
{pergunta}
"""

# 🧩 Define o objeto de prompt usando o LangChain
prompt = PromptTemplate(
    input_variables=["contexto", "pergunta"],  # Variáveis que serão substituídas dinamicamente
    template=template                          # Estrutura textual base
)

# 🔗 Constrói a cadeia LLMChain: conecta o modelo (llm) ao prompt (template)
# Essa cadeia recebe entrada (pergunta + contexto) e retorna a resposta gerada
chain = LLMChain(llm=llm, prompt=prompt)

# 💡 Essa arquitetura leve baseada em LLMChain é mais compatível com a API Gemini do que o uso de agentes tradicionais (AgentExecutor), que geram erros de campo ("thought", "action", etc.).


C:\Users\Usuário\AppData\Local\Temp\ipykernel_11588\3466358861.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [5]:
# ✅ O agente lê e seleciona os dados, pronto para consultas.

# 📦 Importa a biblioteca padrão para manipulação de arquivos ZIP
import zipfile

# 🔓 Abre o arquivo compactado contendo os CSVs das notas fiscais
with zipfile.ZipFile("data/202401_NFs.zip", 'r') as zip_ref:
    # 📂 Extrai o conteúdo do ZIP para dentro da pasta destino especificada
    zip_ref.extractall("data/202401_NFs/")


In [6]:
# ✅ Leitura dos arquivos CSV

# 📄 Carrega o arquivo CSV contendo os dados gerais das notas fiscais (cabeçalho: fornecedor, valor, data etc.)
df_cabecalho = pd.read_csv('data/202401_NFs/202401_NFs_Cabecalho.csv')

# 📦 Carrega o arquivo CSV contendo os dados detalhados dos itens das notas (descrição, quantidade, valor unitário etc.)
df_itens = pd.read_csv('data/202401_NFs/202401_NFs_Itens.csv')

# 📊 Exibe a dimensão dos DataFrames carregados (linhas x colunas) no terminal, com estilo visual
console.print(
    Panel(
        f"📄 Cabeçalho: {df_cabecalho.shape}\n📦 Itens: {df_itens.shape}",
        title="Arquivos CSV carregados"
    )
)

# 🔍 Dica: .shape retorna uma tupla (n_linhas, n_colunas) — útil para validação rápida da integridade dos dados após a leitura.


╭──────────────────────────────────────────── Arquivos CSV carregados ────────────────────────────────────────────╮
│ 📄 Cabeçalho: (100, 21)                                                                                         │
│ 📦 Itens: (565, 27)                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
# 🔁 Loop contínuo para interação com o usuário
while True:
    # 🟢 Entrada de uma pergunta do usuário via terminal
    pergunta = input("🟢 Digite sua pergunta (ou 'sair'): ").strip()
    
    # 🔚 Condição de saída: se o usuário digitar 'sair', encerra o agente
    if pergunta.lower() == "sair":
        print("👋 Encerrando o agente fiscal.")
        break

    # 📄 Geração de contexto para o modelo de linguagem
    # Aqui são usadas as 3 primeiras linhas de cada DataFrame (cabeçalho e itens)
    # Isso fornece ao modelo um "resumo" da estrutura dos dados reais
    contexto = (
        f"Cabecalho:\n{df_cabecalho.head(3).to_string(index=False)}"
        f"\n\nItens:\n{df_itens.head(3).to_string(index=False)}"
    )

    try:
        # 🤖 Executa a cadeia LLMChain passando o contexto e a pergunta do usuário
        resposta = chain.run({"contexto": contexto, "pergunta": pergunta})
        
        # ✅ Exibe a resposta em um painel estilizado usando a biblioteca Rich
        console.print(Panel(resposta, title="📢 Resposta do agente", border_style="green"))

    except Exception as e:
        # ❌ Exibe mensagem de erro estilizada caso a execução falhe
        console.print(Panel(f"[red]Erro:[/red] {e}", title="❌ Falha"))


╭───────────────────────────────────────────── 📢 Resposta do agente ─────────────────────────────────────────────╮
│ Com base nos dados fornecidos, a **COMPANHIA BRASILEIRA DE EDUC. E SIST. DE ENS. S.A - PR OP** recebeu o maior  │
│ montante, com R$ 522,50.  Embora a segunda nota fiscal tenha um valor total maior (R$ 499,00),  a primeira nota │
│ fiscal, emitida pela COMPANHIA BRASILEIRA DE EDUC. E SIST. DE ENS. S.A - PR OP,  é superior.                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── 📢 Resposta do agente ─────────────────────────────────────────────╮
│ O item com maior quantidade entregue foi o "CINEMATICO RODO-AR" (item 2 da segunda nota fiscal), com 4 peças.   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── 📢 Resposta do agente ─────────────────────────────────────────────╮
│ Nenhuma nota fiscal foi emitida em 2024-01-10.  Os dados fornecidos mostram notas emitidas em 2024-01-18,       │
│ 2024-01-26 e 2024-01-22.                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── 📢 Resposta do agente ─────────────────────────────────────────────╮
│ A média do valor das notas fiscais emitidas é de R$ 453,50.  Este valor foi calculado somando os valores das    │
│ três notas (522,5 + 499,0 + 337,5 = 1359,0) e dividindo pelo número de notas (3).                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

👋 Encerrando o agente fiscal.
